In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
5,Covid-19 Live Updates: Health Officials Tiptoe...,https://www.nytimes.com/2020/09/20/world/covid...,"Mon, 21 Sep 2020 03:21:00 GMT",2020-09-21 03:21:00+00:00
0,Most of the US is headed in the wrong directio...,https://www.cnn.com/2020/09/20/health/us-coron...,"Mon, 21 Sep 2020 00:10:00 GMT",2020-09-21 00:10:00+00:00
7,Tracking COVID-19 in Alaska: 95 new cases repo...,https://www.adn.com/alaska-news/2020/09/20/tra...,"Sun, 20 Sep 2020 22:37:25 GMT",2020-09-20 22:37:25+00:00
3,"COVID Testing Of Teachers, Staff Hotly Debated...",http://www.capradio.org/articles/2020/09/20/co...,"Sun, 20 Sep 2020 22:18:00 GMT",2020-09-20 22:18:00+00:00
9,Latest on COVID-19 in MN: State sees record nu...,https://www.mprnews.org/story/2020/09/20/lates...,"Sun, 20 Sep 2020 16:28:00 GMT",2020-09-20 16:28:00+00:00
4,Governor Cuomo Updates New Yorkers on State's ...,https://www.governor.ny.gov/news/governor-cuom...,"Sun, 20 Sep 2020 15:00:21 GMT",2020-09-20 15:00:21+00:00
1,COVID-19 Daily Update 9-20-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sun, 20 Sep 2020 14:50:53 GMT",2020-09-20 14:50:53+00:00
6,COVID-19 Data Review Panel Examines States Dat...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sat, 19 Sep 2020 21:10:19 GMT",2020-09-19 21:10:19+00:00
2,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Fri, 18 Sep 2020 21:10:00 GMT",2020-09-18 21:10:00+00:00
8,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Thu, 17 Sep 2020 21:30:00 GMT",2020-09-17 21:30:00+00:00
